# Song recommender

In [1]:
#Importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import json_normalize
from pickle import load
import numpy as np
from sklearn.preprocessing import StandardScaler
from pickle import dump
from sklearn.cluster import KMeans

In [2]:
# Uploading top 100 data frame
hot_songs = pd.read_csv('hot_songs.csv')
hot_songs

,title,artist
0,I Had Some Help (feat. Morgan Wallen),Post Malone
1,A Bar Song (Tipsy),Shaboozey
2,Not Like Us,Kendrick Lamar
3,Lose Control,Teddy Swims
4,HIND'S HALL,Macklemore
...,...,...
95,Chevrolet (feat. Jelly Roll),Dustin Lynch
96,Broadway Girls (feat. Morgan Wallen),Lil Durk
97,Devil You Know,Tyler Braden
98,Lose My Breath (Instrumental),Stray Kids & Charlie Puth


In [3]:
# Uploading +10,000 song playlist
df_artists_ft = pd.read_csv('df_artists_ft.csv')
df_artists_ft

,song_name,name,song_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster_nb
0,...Ready For It?,Taylor Swift,7zgqtptZvhf8GEmdsM2vp2,0,0.615,0.779,2.0,-6.454,1.0,0.1350,...,0.453,160.000,audio_features,7zgqtptZvhf8GEmdsM2vp2,spotify:track:7zgqtptZvhf8GEmdsM2vp2,https://api.spotify.com/v1/tracks/7zgqtptZvhf8...,https://api.spotify.com/v1/audio-analysis/7zgq...,208198.0,4.0,7
1,Life Changes,Thomas Rhett,4Vxu50qVrQcycjRyJQaZLC,59,0.687,0.845,7.0,-4.370,1.0,0.0575,...,0.809,87.972,audio_features,4Vxu50qVrQcycjRyJQaZLC,spotify:track:4Vxu50qVrQcycjRyJQaZLC,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,https://api.spotify.com/v1/audio-analysis/4Vxu...,190227.0,4.0,1
2,24K Magic,Bruno Mars,6b8Be6ljOzmkOmFslEb23P,81,0.818,0.803,1.0,-4.282,1.0,0.0797,...,0.632,106.970,audio_features,6b8Be6ljOzmkOmFslEb23P,spotify:track:6b8Be6ljOzmkOmFslEb23P,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,https://api.spotify.com/v1/audio-analysis/6b8B...,225983.0,4.0,1
3,Galway Girl,Ed Sheeran,0afhq8XCExXpqazXczTSve,78,0.624,0.876,9.0,-3.374,1.0,0.1000,...,0.781,99.943,audio_features,0afhq8XCExXpqazXczTSve,spotify:track:0afhq8XCExXpqazXczTSve,https://api.spotify.com/v1/tracks/0afhq8XCExXp...,https://api.spotify.com/v1/audio-analysis/0afh...,170827.0,4.0,1
4,Photograph,Ed Sheeran,1HNkqx9Ahdgi1Ixy2xkKkL,83,0.614,0.379,4.0,-10.480,1.0,0.0476,...,0.201,107.989,audio_features,1HNkqx9Ahdgi1Ixy2xkKkL,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi...,https://api.spotify.com/v1/audio-analysis/1HNk...,258987.0,4.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12131,Fool's Overture,Supertramp,5pSSEkT0963muzzIjsVkrs,51,0.406,0.306,3.0,-10.482,1.0,0.0372,...,0.073,135.272,audio_features,5pSSEkT0963muzzIjsVkrs,spotify:track:5pSSEkT0963muzzIjsVkrs,https://api.spotify.com/v1/tracks/5pSSEkT0963m...,https://api.spotify.com/v1/audio-analysis/5pSS...,652560.0,4.0,6
12132,Heart of the Sunrise - 2003 Remaster,Yes,7gC6Rbllqf1yXNC02e5jz2,44,0.362,0.507,1.0,-11.229,1.0,0.0394,...,0.456,146.641,audio_features,7gC6Rbllqf1yXNC02e5jz2,spotify:track:7gC6Rbllqf1yXNC02e5jz2,https://api.spotify.com/v1/tracks/7gC6Rbllqf1y...,https://api.spotify.com/v1/audio-analysis/7gC6...,634440.0,3.0,2
12133,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney,6Ff77WXC58MkhLE5A1qgY1,0,0.331,0.733,2.0,-8.671,1.0,0.0468,...,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0,7
12134,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings,6Ff77WXC58MkhLE5A1qgY1,0,0.331,0.733,2.0,-8.671,1.0,0.0468,...,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0,7


In [4]:
#Authentication to spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
secrets_file = open("secrets.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        
#authentication with secrets
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [5]:
#First prototype
#Limitations: it only collects song title, and needs to be written as it is found in the hot songs list (respecting cap letters) 
import random

def proposed_title_first_prot(top_100):
    input_title = input("Enter a song title: ")
    input_title = input_title.strip()
    if input_title in top_100['title'].values:
        proposal = top_100.sample()
        print('Try this top 100 song:')
        print(proposal['title'].values[0]+' from '+proposal['artist'].values[0])
    else:
        print("Sorry I don't have a top 100 song recommendation at this time")         
    return


In [6]:
#Testing 1st prototype (Scenario: getting a top 100 recommendation)

proposed_title_first_prot(hot_songs)  

Enter a song title: Lose Control
Try this top 100 song:
A Bar Song (Tipsy) from Shaboozey


In [7]:
#Testing 1st prototype (Scenario: not getting a recommendation)

proposed_title_first_prot(hot_songs)  

Enter a song title: melody
Sorry I don't have a top 100 song recommendation at this time


In [8]:
#Second prototype
#Limitations: to get a top_100 recommendation inputted song needs to be written as it is found in the hot songs list
#Limitations: it only collects (for now), song title, not artist 
def proposed_title_second_prot(top_100, top_10000):
    input_title = input("Enter a song title: ")
    input_title = input_title.strip()
    #Collecting from Spotify inputted song's overall info
    results = sp.search(q=input_title, limit=1)
    #Checking if it's in top_100 list
    if input_title in top_100['title'].values:
        proposal = top_100.sample()
        print('Try this top 100 song:')
        print(proposal['title'].values[0]+' from '+proposal['artist'].values[0])
    #If not in top_100 list, then searching for a cluster recommendation
    elif results['tracks']['items'] != []:
        #Extracting only those needed ('href', 'id', 'name', 'type', 'uri', 'external_urls.spotify', 'song_id', 'song_name')
        tracks = json_normalize(results["tracks"]["items"])
        artists_for_song = json_normalize(tracks.iloc[0]['artists'])
        artists_for_song['song_id'] = tracks.iloc[0]['id']
        input_song_id = artists_for_song['song_id'][0]
        #Collecting from Spotify inputted song's features
        song_ft = sp.audio_features(input_song_id)
        #Storing them in variable 'song_audio_features'
        song_audio_features = json_normalize(song_ft)
        #Extracting only its numericals
        song_audio_features = song_audio_features.select_dtypes(np.number).reset_index(drop=True)
        #Loading scaler
        scaler = load(open('scaler.pkl', 'rb'))
        #Scaling song
        song_scaled = scaler.transform(song_audio_features)
        #loading model
        kmeans = load(open('kmeans.pkl', 'rb'))
        #Predicting cluster
        cluster = int(kmeans.predict(song_scaled))
        #Proposing song
        proposal = top_10000[top_10000['cluster_nb'] == cluster]                                                                         
        proposal = proposal.sample()
        print('Try this recommendation from a Spotify playlist:')
        print(proposal['song_name'].values[0]+' from '+proposal['name'].values[0])
    #If introduced title is not found in Spotify, printing
    else:
        print("Sorry I can't provide you with a recommendation. Check for typos or enter a song again")         
    return

In [9]:
#Testing 2nd protype (scenario: getting a top 100 reco)
proposed_title_second_prot(hot_songs, df_artists_ft)

Enter a song title: Lose Control
Try this top 100 song:
The Door from Teddy Swims


In [10]:
#Testing 2nd protype (scenario: recommendation from a Spotify playlist)
proposed_title_second_prot(hot_songs, df_artists_ft)

Enter a song title: sweet home alabama
Try this recommendation from a Spotify playlist:
Let's Go Steady from Charlie Brown


In [11]:
#Testing 2nd protype (scenario: not getting a reco)
proposed_title_second_prot(hot_songs, df_artists_ft)

Enter a song title: huilHBDUIBIUHBIU
Sorry I can't provide you with a recommendation. Check for typos or enter a song again
